In [14]:
import os
import pandas as pd
import json
import random
import shutil
import numpy as np

In [15]:
source_pose_dic = './UBNormalAlphaPoseOutput'
source_video_dic = './data/UBnormal'

In [56]:
dst_pose_dic = './STG-NF/data/UBnormal/pose/test'
dst_video_dic = './STG-NF/data/UBnormal/videos'

In [45]:
def get_jsons(directory: str):
    """Function that exctracts the mp4 files from the given directory
    and returns the path to the video and it's annotations
    """
    filelist = [];
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('tracked_person.json'):
                json_file = os.path.join(root,file)
                filelist.append(json_file)
    return filelist

def get_mp4s(directory: str):
    """Function that exctracts the mp4 files from the given directory
    and returns the path to the video and it's annotations
    """
    filelist = [];
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.mp4'):
                json_file = os.path.join(root,file)
                filelist.append(json_file)
    return filelist

def remove_files(directory: str):
    for root, dirs, files in os.walk(directory):
        for file in files:
           file_path = os.path.join(root, file)
           if os.path.isfile(file_path):
             os.remove(file_path)

In [42]:
json_files = get_jsons(source_pose_dic)
mp4_files = get_mp4s(source_video_dic)

In [43]:
mp4_dict = {f.split('/')[-1].split('.')[0]: f for f in mp4_files}

In [50]:
examples_to_be_moved = random.choices(json_files, k=100)

In [70]:
# Avoid issue due to invalid track.txt format
test_scenes = []
while len(test_scenes) < 100:
    scene = random.choices(json_files, k=1)[0]
    pose_file_name = scene.split('/')[-1]
    video_name = '_'.join(pose_file_name.split('-')[0].split('_')[0:-3])
    try:
        np.load(f'./STG-NF/data/UBnormal/gt/{video_name}_tracks.txt', allow_pickle=True)
        if scene not in test_scenes:
            test_scenes.append(scene)
    except:
        pass

In [71]:
remove_files(dst_pose_dic)
remove_files(dst_video_dic)
for pose_file in test_scenes:
    pose_file_name = pose_file.split('/')[-1]
    video_name = '_'.join(pose_file_name.split('-')[0].split('_')[0:-3])
    video_file = mp4_dict[video_name]
    pose_dest = os.path.join(dst_pose_dic, pose_file_name)
    video_dest = os.path.join(dst_video_dic, f'{video_name}.mp4')
    shutil.copyfile(pose_file, pose_dest)
    shutil.copyfile(video_file, video_dest)